In [15]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
#from sklearn.cluster import AgglomerativeClustering
import pandas as pd 


In [16]:
def remove_spaces(stri):
    while stri.startswith(" "):
        stri=stri[1:]

    while stri.endswith(" "):
        stri=stri[:-1]
    return stri
    

In [17]:
df = pd.read_excel("Metrics.xlsx",sheet_name="KPI")
df = df.dropna(axis=0)
for idx in df.index:
    for col in df.columns:
        df.at[idx,col]=remove_spaces(df.at[idx,col])
df.to_csv("metrics_kpi.csv")


In [18]:
cats = [x for x in df.Category.unique()]
dc = ["DC" for x in df.Category.unique()]
group = ["base" for x in df.Category.unique()]
unit = ["u" for x in df.Category.unique()]

#cats.append("Energy")
#cats.append("Clima")


In [19]:
df_network = pd.DataFrame(columns=['Indent Level','Item','Weight','Parent', "Level", "Type"])

#df_network.loc[len(df_network)] = [0,"DC", 1, "", "Facility"]

for i in range(0,len(cats)):
    df_network.loc[len(df_network)] = [0,cats[i], 5, "KPI", "Facility", 1  ]


label = [f"Facility_{c}" for c in cats]    
for c,l in zip(cats,label):
    if c not in ["Energy", "Clima"]:
        df_network.loc[len(df_network)] = [0,l, 2, c, "Facility", 2]

for idx in df_network.index:
    #if df_network.at[idx,"Item"] in ["EnEf", "Green"]:
    #    df_network.at[idx,"Parent"] = "Energy"
    
    #if df_network.at[idx,"Item"] in ["Thermal", "Cooling"]:
    #    df_network.at[idx,"Parent"] = "Clima"

    if df_network.at[idx,"Item"] in ["Network", "Storage"]:
        df_network.at[idx,"Parent"] = "IT Equipment_Performance"
areas = [x for x in df.Area.unique() if x != "Facility"]
areas_total = []
for a in areas:
    for c in cats:
        df2 = df[df["Category"]==c]
        df3 = df2[df2["Area"]==a]
        if df3.shape[0]>0:
            areas_total_label = f"{a}_{c}"
            df_network.loc[len(df_network)] = [0,areas_total_label, 3, f"Facility_{c}" ,a, 3]
            areas_total.append(areas_total_label)

df_network.loc[len(df_network)] =[0,"IT Equipment_Thermal", 3,"Facility_Thermal","IT", 3]
df_network.loc[len(df_network)] =[0,"IT Equipment_Performance", 3,"Facility_Performance", "IT", 3]


for idx in df.index:
    label = f"{df.at[idx,'Area']}_{df.at[idx,'Category']}"
    level = 5
    if df.at[idx,"Area"] != "Facility":
        level = 8
    if df.at[idx,"Area"] in ["Server", "UPS", "Rack", "Storage"]:
        level = 6
    df_network.loc[len(df_network)] = [1,df.at[idx,"Symbol"], 4, label, df.at[idx,"Area"], level ]

for idx in df_network.index:
    for c in cats:
        for device in ["Server", "UPS", "Rack"]: 
            if df_network.at[idx, "Item"] ==f"{device}_{c}":
                df_network.at[idx, "Parent"] = f"IT Equipment_{c}"

for idx in df_network.index:
    if df_network.at[idx, "Parent"] =="Storage_Storage":
                df_network.at[idx, "Parent"] = "Storage"
    if df_network.at[idx, "Parent"] =="IT Equipment_Network":
                df_network.at[idx, "Parent"] = "Network"
    if df_network.at[idx, "Parent"] =="Facility_Network":
                df_network.at[idx, "Parent"] = "Network"
    if df_network.at[idx, "Item"] =="UPS_Performance":
                df_network.at[idx, "Parent"] = "Facility_Performance"

df_network = df_network[df_network["Item"] != 'Facility_Storage']
df_network = df_network[df_network["Item"] != 'Facility_Network']
df_network = df_network[df_network["Item"] != 'Storage_Storage']
df_network = df_network[df_network["Item"] != 'IT Equipment_Network']

for idx in df_network.index:
    df_network.at[idx, "Item"] =df_network.at[idx, "Item"].replace("Facility_","")
    df_network.at[idx, "Parent"] =df_network.at[idx, "Parent"].replace("Facility_","")
df_network.to_csv("network_data.csv")

In [21]:
import networkx as nx 

# Convert the data into networkx format
G = nx.from_pandas_edgelist(df_network[df_network["Item"] != 'All items'], 
                            source = 'Item', target = 'Parent', edge_attr = 'Weight')

# Set the positions of the nodes in the graph - other options are spring, planar or spectral
pos = nx.kamada_kawai_layout(G, weight = None)

#pos = nx.spring_layout(G)#, weight = None)

financial_kpi = []
for idx in df_network.index:
    if df_network.at[idx,"Parent"]=="Financial":
        financial_kpi.append(df_network.at[idx,"Item"])

cooling_kpi = []
for idx in df_network.index:
    if df_network.at[idx,"Parent"].endswith("Cooling"):
        cooling_kpi.append(df_network.at[idx,"Item"])

for n, p in pos.items():
    G.nodes[n]['pos'] = p
    if n in financial_kpi:
        G.nodes[n]['pos'] = [p[0]+1/2,p[1]+1/2]

    if n in cooling_kpi:
        G.nodes[n]['pos'] = [p[0]-1/6,p[1]+1/4]

# Add weight and depth attributes to the nodes
for x in df_network["Item"]: 
    G.nodes[x]['Weight'] = df_network[df_network["Item"] == x]["Weight"].values[0]
    G.nodes[x]['Depth'] = df_network[df_network["Item"] == x]["Type"].values[0]

    
# Add position info to edges
edge_x = []
edge_y = []

for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

# Draw the edges
import plotly.graph_objects as go

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []

for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

# Draw the nodes
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='Viridis',
        reversescale=False,
        color=[],
        size=[40-2*x for x in df_network["Type"]],
       
        #colorbar=dict(
        #    thickness=15,
        #    title='Level',
        #    xanchor='left',
        #    titleside='right'
        #),
       
        line_width=0))

node_depth = []
node_text = []

for n in G.nodes.items():
    try:
        node_depth.append(n[1]["Depth"]+5)
    except:
        node_depth.append(0)
    try:
        node_text.append(n[0])# + " {0:.2f}%".format(n[1]["Weight"]))
    except:
        node_text.append("DC")
    if node[0]== "Financial":
        print(n)
node_trace.marker.color = node_depth
node_trace.marker.opacity = [0.5 for x in node_depth]

node_trace.text = node_text

# Draw the full graph
graph = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                #title='<b>Taxonomy of Data Center Indicators<b>',
                #titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)),
                )

texts = [x.split("_")[0] for x in node_text]
texts = [x.replace("IT Equipment", "ICT") for x in texts]
texts = [x.replace("Facility", "DC") for x in texts]

graph.add_trace(go.Scatter(
    x=node_x , y=node_y,
    mode="text",
    name="_",
    text= texts,
    textposition="middle center", 
))

graph.update_layout(
    width=2000*23/10/2,
    height=2000/2,  
    font_size= 20,
    #paper_bgcolor='rgba(0,0,0,0)',
    #plot_bgcolor='rgba(0,0,0,0)'
                  )
graph.show()
#graph.write_image("fig_2024.png")